Goal: Download all student suspension data for all schools in the past 4 years and create a master csv.

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import glob
import os

The export button on the suspension report shows where to specify the year argument in the URL. We wrote a loop to download each year's worth of data to .xls. DESE started publishing suspension data in this format in 2013. 

In [ ]:
years = [2013, 2014, 2015, 2016]

for year in years:
    base = 'http://profiles.doe.mass.edu/state_report/ssdr.aspx?ctl00$ContentPlaceHolder1$fycode={}&export_excel=yes&ctl00$ContentPlaceHolder1$subjectCode=ALL&ctl00$ContentPlaceHolder1$reportType=SCHOOL&ctl00$ContentPlaceHolder1$studentGroup=ALL'
    base = base.format(year)
    print(base)
    filename = "student_"+"suspensions_"+str(year)+".xls"
    with open(filename, 'w') as output:
        output.write(requests.get(base).text)

The below function converts the .xls files to dataframes. Files from 2013 to 2015 are passed through the function and are added to a master dataframe.

In [1]:
def clean_file(file):
    df = pd.read_html(file)
    df = pd.DataFrame(df[1])
    header = df.iloc[0]
    df.columns = header
    df = df[1:]
    file_name = os.path.splitext(file)[0]
    df["Source"] = file_name
    df['Year'] = df["Source"].str[-4:]
    return df

In [13]:
all_files = glob.glob("student_suspensions_201[3-5].xls")
df_list = []

for file in all_files:
    file = clean_file(file)
    df_list.append(file)
    
df = pd.concat(df_list)

Test out mask for suspension data

In [15]:
mask = df["Org Code"] == "04450105"
df_abbyk = df[mask]

df_abbyk

,% Emergency Removal,% Expulsion,% In-School Suspension,% Out-of-School Suspension,% Permanent Expulsion,% Removed to Alternate Setting,Org Code,SCHOOL,Source,Students,Students Disciplined,Year
1,NaN,NaN,0.5,5.5,0.0,0.0,04450105,Abby Kelley Foster Charter Public (District) -...,student_suspensions_2013,1436,83,2013
1,NaN,NaN,0.1,2.4,0.0,0.0,04450105,Abby Kelley Foster Charter Public (District) -...,student_suspensions_2014,1451,35,2014
1,0.0,0.0,0.9,1.9,NaN,0.0,04450105,Abby Kelley Foster Charter Public (District) -...,student_suspensions_2015,1443,35,2015


Set ORG CODE as index and write master file to CSV

In [18]:
df = df.set_index('Org Code')
df.to_csv("student_suspensions_all.csv")